# DeepSeek Prover V2 自动定理证明实验

本 notebook 复现了 DeepSeek 发布的 **DeepSeek-Prover-V2** 自动定理证明系统。

---

## 什么是 DeepSeek Prover V2？

DeepSeek Prover V2 是一个专为 **Lean 4 形式化定理证明** 设计的大语言模型。它能够：
- 理解数学定理的形式化表述
- 生成证明计划（Proof Plan），展示推理思路
- 输出可被 Lean 4 验证的完整证明代码

| 组件 | 功能 | 本实验是否包含 |
|------|------|---------------|
| DeepSeek-Prover-V2-7B | 生成证明计划与 Lean 4 代码 | 包含 |
| DeepSeek-Prover-V2-671B | 更强的推理能力（需要大量 GPU） | 不包含 |
| Lean 4 + Mathlib | 形式化验证证明正确性 | 包含 |

7B 模型在 Colab 免费 GPU（T4）上可以运行，对于基础定理效果较好。复杂的竞赛级题目可能需要 671B 模型或多次重试。

---


# 第一部分：环境配置

In [1]:
#@title 1.1 检查运行环境
#@markdown 检查当前运行环境是否满足要求。

import sys
import subprocess

print("Python 版本:", sys.version)
print()

# 检查 GPU
try:
    import torch
    if torch.cuda.is_available():
        print("GPU 可用:", torch.cuda.get_device_name(0))
        print("显存:", round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 1), "GB")
    else:
        print("警告: 未检测到 GPU，运行速度会很慢")
except ImportError:
    print("PyTorch 未安装，将在下一步安装")

print()
print("环境检查完成")

Python 版本: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

GPU 可用: NVIDIA A100-SXM4-40GB
显存: 39.6 GB

环境检查完成


In [2]:
#@title 1.2 安装依赖与 Lean 4
#@markdown 安装所有必要的依赖：
#@markdown - Python 依赖：Transformers、Accelerate 等
#@markdown - Lean 4：通过 elan 安装
#@markdown - Mathlib 工程：用于 lake build 验证证明
#@markdown
#@markdown 预计时间：5-10 分钟（首次运行）

from google.colab import output
output.enable_custom_widget_manager()

import subprocess
import os

os.environ["PATH"] = os.path.expanduser("~/.elan/bin") + ":" + os.environ.get("PATH", "")

cmd = r"""
set -euo pipefail

pip -q install -U transformers accelerate sentencepiece ipywidgets

apt-get -qq update
apt-get -qq install -y curl git zstd

if [ ! -d "$HOME/.elan" ]; then
  curl -sSf https://elan.lean-lang.org/elan-init.sh | sh -s -- -y
fi
export PATH="$HOME/.elan/bin:$PATH"

mkdir -p /content/lean_ds
cd /content/lean_ds

if [ -d ds_demo ]; then
  echo "ds_demo 已存在，跳过创建"
else
  lake +leanprover-community/mathlib4:lean-toolchain new ds_demo math
fi

cd ds_demo
lake exe cache get

echo "安装完成"
"""

print("正在安装依赖，请稍候...")
print()
p = subprocess.run(["bash", "-lc", cmd], capture_output=True, text=True)

if p.returncode != 0:
    print("===== STDOUT =====")
    print(p.stdout)
    print("\n===== STDERR =====")
    print(p.stderr)
    raise RuntimeError("安装失败，请检查上方日志")
else:
    print("依赖安装完成")

正在安装依赖，请稍候...

依赖安装完成


In [3]:
#@title 1.3 验证安装
#@markdown 验证 Lean 4 和相关工具是否安装成功。

import subprocess
import os

os.environ["PATH"] = os.path.expanduser("~/.elan/bin") + ":" + os.environ.get("PATH", "")

print("验证 Lean 4 安装:")
print("-" * 40)

# 检查 lean
r = subprocess.run(["lean", "--version"], capture_output=True, text=True)
if r.returncode == 0:
    print("[OK] Lean:", r.stdout.strip().split("\n")[0])
else:
    print("[FAIL] Lean 未安装")

# 检查 lake
r = subprocess.run(["lake", "--version"], capture_output=True, text=True)
if r.returncode == 0:
    print("[OK] Lake:", r.stdout.strip().split("\n")[0])
else:
    print("[FAIL] Lake 未安装")

# 检查工程目录
if os.path.exists("/content/lean_ds/ds_demo"):
    print("[OK] Mathlib 工程目录存在")
else:
    print("[FAIL] Mathlib 工程目录不存在")

print("-" * 40)
print("验证完成")

验证 Lean 4 安装:
----------------------------------------
[OK] Lean: Lean (version 4.27.0, x86_64-unknown-linux-gnu, commit db93fe1608548721853390a10cd40580fe7d22ae, Release)
[OK] Lake: Lake version 5.0.0-src+db93fe1 (Lean version 4.27.0)
[OK] Mathlib 工程目录存在
----------------------------------------
验证完成


# 第二部分：模型配置

## 参数说明：模型设置

| 参数 | 说明 | 建议值 |
|------|------|--------|
| `MODEL_ID` | HuggingFace 模型路径 | Colab T4 使用 7B 版本 |
| `jobname` | 实验名称，用于保存输出文件 | 如 `zhangsan_exp1` |

In [13]:
#@title 2.1 设置模型与实验名称

import os
import re
import time

MODEL_ID = "deepseek-ai/DeepSeek-Prover-V2-7B"  #@param {type:"string"}
jobname = "my_experiment"  #@param {type:"string"}

safe_job = re.sub(r"[^a-zA-Z0-9_\-]+", "_", jobname).strip("_") or "job"
jobdir = f"/content/ds_prover_runs/{safe_job}_{int(time.time())}"
os.makedirs(jobdir, exist_ok=True)

print("模型配置完成")
print("  模型:", MODEL_ID)
print("  输出目录:", jobdir)

模型配置完成
  模型: deepseek-ai/DeepSeek-Prover-V2-7B
  输出目录: /content/ds_prover_runs/my_experiment_1769503401


In [14]:
#@title 2.2 加载模型
#@markdown 下载并加载 DeepSeek Prover V2 模型。
#@markdown
#@markdown 预计时间：
#@markdown - 首次运行：10-15 分钟（下载约 14GB）
#@markdown - 后续运行：1-2 分钟（从缓存加载）

from google.colab import output
output.enable_custom_widget_manager()


import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

import transformers
transformers.logging.set_verbosity_error()

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

SEED = 42
torch.manual_seed(SEED)

if torch.cuda.is_available():
    major = torch.cuda.get_device_capability(0)[0]
    dtype = torch.bfloat16 if major >= 8 else torch.float16
    print("GPU:", torch.cuda.get_device_name(0))
else:
    dtype = torch.float16
    print("警告: 未检测到 GPU")

print("正在加载模型:", MODEL_ID)
print("首次运行需下载约 14GB，请耐心等待...")
print()

tok = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=dtype,
    trust_remote_code=True
)

print()
print("模型加载完成")
print("  dtype:", dtype)
print("  device:", model.device)

GPU: NVIDIA A100-SXM4-40GB
正在加载模型: deepseek-ai/DeepSeek-Prover-V2-7B
首次运行需下载约 14GB，请耐心等待...



Loading weights:   0%|          | 0/273 [00:00<?, ?it/s]


模型加载完成
  dtype: torch.bfloat16
  device: cuda:0


# 第三部分：运行证明

## 参数说明：生成参数

| 参数 | 说明 | 建议范围 |
|------|------|----------|
| `MAX_NEW_TOKENS` | 生成长度上限 | 1200-2000（越大越慢） |
| `TEMPERATURE` | 采样随机性 | 0.3-0.7（越低越稳定） |
| `TOP_P` | 核采样概率 | 0.9-0.98 |
| `NUM_TRIES` | 验证失败时重试次数 | 3-5 |

**调参建议：**
- 想要更稳定的输出：降低 `TEMPERATURE` 到 0.3-0.5
- 想要更快的生成：降低 `MAX_NEW_TOKENS` 到 1200
- 想要更多探索：提高 `NUM_TRIES` 到 5

In [15]:
#@title 3.1 设置生成参数

MAX_NEW_TOKENS = 2000  #@param {type:"integer"}
TEMPERATURE = 0.6      #@param {type:"number"}
TOP_P = 0.95           #@param {type:"number"}
NUM_TRIES = 3          #@param {type:"integer"}

print("生成参数已设置")
print("  MAX_NEW_TOKENS =", MAX_NEW_TOKENS)
print("  TEMPERATURE    =", TEMPERATURE)
print("  TOP_P          =", TOP_P)
print("  NUM_TRIES      =", NUM_TRIES)

生成参数已设置
  MAX_NEW_TOKENS = 2000
  TEMPERATURE    = 0.6
  TOP_P          = 0.95
  NUM_TRIES      = 3


## 参数说明：输入定理

在**3.2 输入定理**文本框中直接修改内容。

格式要求：
- 以 `import Mathlib` 开头
- 包含 `sorry` 占位符
- 定理用 `theorem xxx : ... := by` 格式

| 部分 | 是否需要改 | 说明 |
|------|-----------|------|
| `import Mathlib` | 通常不改 | 固定写法，导入数学库 |
| `import Aesop` | 通常不改 | 固定写法，导入自动证明策略 |
| `set_option maxHeartbeats 0` | 通常不改 | 固定写法，取消计算时间限制 |
| `open ...` | 通常不改 | 固定写法，打开命名空间 |
| `/-- 描述 -/` | 可以改 | 用自然语言描述你的定理 |
| `theorem 名称` | 需要改 | 给你的定理起个名字 |
| `: 命题内容` | 需要改 | 写你要证明的数学式子 |
| `sorry` | 不要改 | AI 会自动替换成证明 |

**示例：**
```lean
/-- 证明 1 + 1 = 2 -/
theorem one_plus_one : 1 + 1 = 2 := by
  sorry

/-- 绝对值 -/
theorem mathd_algebra_10 : abs ((120 : Real) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry

/-- 不等式 -/
theorem simple_ineq (x : Real) (h : x > 0) : x + 1 > 1 := by
  sorry
```

In [7]:
# ================================================================
# 符号速查卡
# ================================================================

#@title  符号输入指南
#@markdown 你可以用简单的 ASCII 输入，系统会自动转换为 Lean 4 符号

from IPython.display import display, Markdown

md = r"""
##  Lean 4 符号速查卡

###  数学类型
| 你输入 | 转换为 | 含义 |
|:------:|:------:|:----:|
| `Real` | `ℝ` | 实数 |
| `Rat` | `ℚ` | 有理数 |
| `Int` | `ℤ` | 整数 |
| `Nat` | `ℕ` | 自然数 |

###  绝对值
| 你输入 | 转换为 |
|:------:|:------:|
| `abs(x)` | `|x|` |
| `abs(x - y)` | `|x - y|` |

###  逻辑符号
| 你输入 | 转换为 | 含义 |
|:------:|:------:|:----:|
| `forall` | `∀` | 对于所有 |
| `exists` | `∃` | 存在 |
| `->` | `→` | 蕴含/函数 |
| `<->` | `↔` | 当且仅当 |

###  比较运算
| 你输入 | 转换为 | 含义 |
|:------:|:------:|:----:|
| `<=` | `≤` | 小于等于 |
| `>=` | `≥` | 大于等于 |
| `!=` | `≠` | 不等于 |

---
**示例：** `theorem t : forall x : Real, abs(x) >= 0`
**转换：** `theorem t : ∀ x : ℝ, |x| ≥ 0`
"""

display(Markdown(md))


##  Lean 4 符号速查卡

###  数学类型
| 你输入 | 转换为 | 含义 |
|:------:|:------:|:----:|
| `Real` | `ℝ` | 实数 |
| `Rat` | `ℚ` | 有理数 |
| `Int` | `ℤ` | 整数 |
| `Nat` | `ℕ` | 自然数 |

###  绝对值
| 你输入 | 转换为 |
|:------:|:------:|
| `abs(x)` | `|x|` |
| `abs(x - y)` | `|x - y|` |

###  逻辑符号
| 你输入 | 转换为 | 含义 |
|:------:|:------:|:----:|
| `forall` | `∀` | 对于所有 |
| `exists` | `∃` | 存在 |
| `->` | `→` | 蕴含/函数 |
| `<->` | `↔` | 当且仅当 |

###  比较运算
| 你输入 | 转换为 | 含义 |
|:------:|:------:|:----:|
| `<=` | `≤` | 小于等于 |
| `>=` | `≥` | 大于等于 |
| `!=` | `≠` | 不等于 |

---
**示例：** `theorem t : forall x : Real, abs(x) >= 0`
**转换：** `theorem t : ∀ x : ℝ, |x| ≥ 0`


In [16]:
#@title 3.2 输入定理
#@markdown 在下方文本框中编辑代码，点击「保存」按钮，继续运行下一步。

from IPython.display import display, HTML
from google.colab import output

LEAN_CODE = """import Mathlib
import Aesop

set_option maxHeartbeats 0
open BigOperators Real Nat Topology Rat

/-- 不等式 -/
theorem simple_ineq (x : Real) (h : x > 0) : x + 1 > 1 := by
  sorry
"""

def save_lean(code):
    global LEAN_CODE
    LEAN_CODE = code
    print(f"已保存，共 {len(code.splitlines())} 行")
    if "sorry" in code:
        print("检测到 sorry，AI 将尝试替换为证明")
    else:
        print("警告: 未检测到 sorry")

output.register_callback('save_lean', save_lean)

html_code = f"""
<textarea id="lean_input" style="width:100%; height:300px; font-family:monospace; font-size:14px;">{LEAN_CODE}</textarea>
<br><br>
<button onclick="google.colab.kernel.invokeFunction('save_lean', [document.getElementById('lean_input').value], {{}})"
        style="padding:8px 16px; background:#4CAF50; color:white; border:none; border-radius:4px; cursor:pointer;">
  保存
</button>
"""

display(HTML(html_code))

已保存，共 9 行
检测到 sorry，AI 将尝试替换为证明


In [18]:
#@title 3.3 生成证明并验证
#@markdown 核心步骤：
#@markdown 1. 预处理：自动转换学生输入的 ASCII 符号为 Lean 4 Unicode
#@markdown 2. 让 AI 生成证明计划（Proof Plan）和 Lean 4 代码
#@markdown 3. 后处理：自动修复常见语法问题
#@markdown 4. 写入文件并运行 lake build 验证
#@markdown 5. 如果失败，智能分析错误并回喂模型修复

import os
import re
import glob
import subprocess
import time
import torch
from typing import Dict
from IPython.display import display, Markdown

os.environ["PATH"] = os.path.expanduser("~/.elan/bin") + ":" + os.environ.get("PATH", "")

# =========================
# 输入预处理器
# =========================

TYPE_MAPPINGS = {
    r'\bReal\b': 'ℝ', r'\bRat\b': 'ℚ', r'\bInt\b': 'ℤ',
    r'\bNat\b': 'ℕ', r'\bComplex\b': 'ℂ',
    r':\s*real\b': ': ℝ', r':\s*rat\b': ': ℚ',
    r':\s*int\b': ': ℤ', r':\s*nat\b': ': ℕ',
}

LOGIC_MAPPINGS = {
    r'\bforall\b': '∀', r'\bexists\b': '∃', r'\blam\b': 'λ',
}

OPERATOR_MAPPINGS = {
    r'<->': '↔', r'(?<!\|)->': '→', r'<=': '≤', r'>=': '≥', r'!=': '≠',
}

def convert_abs_to_bars(code: str) -> str:
    """将 abs(...) 转换为 |...|"""
    result = []
    i = 0
    while i < len(code):
        if code[i:i+4].lower() == 'abs(':
            paren_count = 1
            start = i + 4
            j = start
            while j < len(code) and paren_count > 0:
                if code[j] == '(':
                    paren_count += 1
                elif code[j] == ')':
                    paren_count -= 1
                j += 1
            if paren_count == 0:
                inner = code[start:j-1]
                inner = convert_abs_to_bars(inner)
                result.append(f'|{inner}|')
                i = j
            else:
                result.append(code[i])
                i += 1
        else:
            result.append(code[i])
            i += 1
    return ''.join(result)

def apply_mappings(code: str, mappings: Dict[str, str]) -> str:
    for pattern, replacement in mappings.items():
        code = re.sub(pattern, replacement, code)
    return code

def preprocess_student_input(code: str) -> str:
    """预处理学生输入：ASCII -> Unicode"""
    code = convert_abs_to_bars(code)
    code = apply_mappings(code, TYPE_MAPPINGS)
    code = apply_mappings(code, LOGIC_MAPPINGS)
    code = apply_mappings(code, OPERATOR_MAPPINGS)
    return code

# =========================
# 代码后处理器
# =========================

def postprocess_lean(code: str) -> str:
    """后处理 AI 生成的代码"""
    # 确保有 Mathlib 导入
    if not re.search(r'import\s+Mathlib', code):
        code = "import Mathlib\n\n" + code

    # 替换 abs() 为 ||
    code = convert_abs_to_bars(code)

    # 去重导入
    lines = code.split('\n')
    seen_imports = set()
    new_lines = []
    for line in lines:
        stripped = line.strip()
        if stripped.startswith('import '):
            if stripped not in seen_imports:
                seen_imports.add(stripped)
                new_lines.append(line)
        else:
            new_lines.append(line)

    # 确保 import 在最前面
    import_lines = [l for l in new_lines if l.strip().startswith('import ')]
    other_lines = [l for l in new_lines if not l.strip().startswith('import ')]
    while other_lines and not other_lines[0].strip():
        other_lines.pop(0)

    return '\n'.join(import_lines) + '\n\n' + '\n'.join(other_lines)

# =========================
# 定位 Lean 工程目录
# =========================

def find_project_dir():
    for path in ["/content/lean_ds/ds_demo/lakefile.toml",
                 "/content/lean_ds/ds_demo/lakefile.lean"]:
        if os.path.exists(path):
            return os.path.dirname(path)
    raise FileNotFoundError("找不到 Lean 工程，请先运行第一部分")

PROJECT_DIR = find_project_dir()

def snake_to_camel(s):
    parts = re.split(r"[_\-\s]+", s.strip())
    return "".join(p[:1].upper() + p[1:] for p in parts if p)

proj_name = os.path.basename(PROJECT_DIR)
lib_dir = snake_to_camel(proj_name)
lib_path = os.path.join(PROJECT_DIR, lib_dir)
if not os.path.isdir(lib_path):
    lib_path = PROJECT_DIR

OUT_LEAN = os.path.join(lib_path, "Demo.lean")
os.makedirs(os.path.dirname(OUT_LEAN), exist_ok=True)

# 确保 Demo.lean 被编译
entry_file = os.path.join(PROJECT_DIR, f"{lib_dir}.lean")
if os.path.abspath(lib_path) != os.path.abspath(PROJECT_DIR) and os.path.exists(entry_file):
    imp = f"import {lib_dir}.Demo"
    txt = open(entry_file, "r", encoding="utf-8").read()
    if imp not in txt:
        with open(entry_file, "w", encoding="utf-8") as f:
            f.write(imp + "\n" + txt)
        print(f"已添加 import 到 {lib_dir}.lean")

print("=" * 60)
print("Lean 4 自动证明系统")
print("=" * 60)
print(f"Lean 工程目录: {PROJECT_DIR}")
print(f"输出文件路径: {OUT_LEAN}")

# =========================
# Prompt 模板（关键：保持英文，只要求证明计划用中文）
# =========================

PROMPT_TEMPLATE = """
Complete the following Lean 4 code:
```lean4
{lean}
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan in Chinese (中文).

Please explain your proof strategy in Chinese, including:
1. Problem understanding: What does this theorem state?
2. Mathematical background: What concepts are involved?
3. Proof strategy: What approach will you use?
4. Key steps: Step-by-step reasoning
5. Lean tactics: Which tactics will you use?

IMPORTANT CODE REQUIREMENTS:
- The Lean 4 code must be syntactically correct
- Use |x| notation for absolute value, NOT abs(x)
- Specify numeric types like (n : ℚ) or (n : ℝ)
- Do NOT include Chinese characters in the code
- Do NOT include Chinese comments in the code
- Only the proof plan should be in Chinese

Finally, provide the complete Lean 4 proof code.
""".strip()

REPAIR_PROMPT_TEMPLATE = """
The following Lean 4 code failed to compile. Please fix it.

Requirements:
- Keep the theorem statement unchanged
- Replace all sorry with valid proofs
- Use |x| notation for absolute value, NOT abs(x)
- Do NOT include Chinese characters or comments in the code
- First provide a brief analysis of the error (in Chinese is OK)
- Then output the corrected Lean 4 code

Compiler error:
{error_msg}

Code:
```lean4
{lean_src}
```
""".strip()

# =========================
# 辅助函数
# =========================

def extract_lean_block(txt):
    m = re.search(r"```(?:lean4|lean)\s*(.*?)```", txt, flags=re.S)
    return m.group(1).strip() if m else txt.strip()

def extract_proof_plan(txt):
    m = re.search(r"```(?:lean4|lean)", txt)
    if m:
        return txt[:m.start()].strip()
    return ""

def brief_errors(log, tail=100):
    lines = log.splitlines()
    err = [ln for ln in lines if "error:" in ln.lower() or "unknown" in ln.lower()]
    return "\n".join(err[:40]) if err else "\n".join(lines[-tail:])

# =========================
# 生成函数
# =========================

if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token

def generate_proof(lean_src, is_fix=False, error_msg=""):
    if is_fix:
        prompt = REPAIR_PROMPT_TEMPLATE.format(error_msg=error_msg, lean_src=lean_src)
    else:
        prompt = PROMPT_TEMPLATE.format(lean=lean_src)

    chat = [{"role": "user", "content": prompt}]
    inputs = tok.apply_chat_template(
        chat, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    )

    # 兼容不同版本的 transformers
    if isinstance(inputs, torch.Tensor):
        input_ids = inputs.to(model.device)
    else:
        input_ids = inputs["input_ids"].to(model.device)

    attention_mask = torch.ones_like(input_ids)

    out = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=MAX_NEW_TOKENS,
        do_sample=(TEMPERATURE > 0),
        temperature=TEMPERATURE if TEMPERATURE > 0 else 1.0,
        top_p=TOP_P,
        pad_token_id=tok.eos_token_id,
    )

    full_output = tok.decode(out[0], skip_special_tokens=True)
    input_text = tok.decode(input_ids[0], skip_special_tokens=True)
    generated = full_output[len(input_text):].strip()
    return generated

def write_lean(src):
    processed = postprocess_lean(src)
    with open(OUT_LEAN, "w", encoding="utf-8") as f:
        f.write(processed.strip() + "\n")
    return processed

def lake_build():
    cmd = f'export PATH="$HOME/.elan/bin:$PATH" && cd "{PROJECT_DIR}" && lake build'
    p = subprocess.run(["bash", "-c", cmd], capture_output=True, text=True)
    log = (p.stdout or "") + "\n" + (p.stderr or "")
    return (p.returncode == 0), log

# =========================
# 主流程
# =========================

os.makedirs(jobdir, exist_ok=True)

# 【预处理】转换学生输入
print()
print("=" * 60)
print("【预处理】转换输入符号")
print("=" * 60)

original_input = LEAN_CODE
processed_input = preprocess_student_input(LEAN_CODE)

if processed_input != original_input:
    print("✓ 已自动转换以下符号：")
    print("-" * 40)
    print("原始输入（部分）:")
    # 只显示不同的部分
    for orig_line, proc_line in zip(original_input.split('\n'), processed_input.split('\n')):
        if orig_line != proc_line:
            print(f"  {orig_line}")
            print(f"  → {proc_line}")
    print("-" * 40)
else:
    print("✓ 输入无需转换")

LEAN_CODE = processed_input

with open(os.path.join(jobdir, "input_original.txt"), "w", encoding="utf-8") as f:
    f.write(original_input)
with open(os.path.join(jobdir, "input_processed.txt"), "w", encoding="utf-8") as f:
    f.write(LEAN_CODE)

# 【第一步】生成证明
print()
print("=" * 60)
print("【第一步】生成证明计划与代码")
print("=" * 60)

t0 = time.time()
full_response = generate_proof(LEAN_CODE)
print(f"✓ 生成完成，用时 {time.time()-t0:.1f}s")

proof_plan = extract_proof_plan(full_response)
current = extract_lean_block(full_response)

with open(os.path.join(jobdir, "full_response_1.txt"), "w", encoding="utf-8") as f:
    f.write(full_response)
with open(os.path.join(jobdir, "candidate_1.lean"), "w", encoding="utf-8") as f:
    f.write(current + "\n")

if proof_plan:
    print()
    print("【证明计划 (Proof Plan)】")
    print("-" * 40)
    display(Markdown(proof_plan))
    print("-" * 40)

# 验证循环
ok = False
last_log = ""

for attempt in range(1, NUM_TRIES + 1):
    print()
    print("=" * 60)
    print(f"【第二步】lake build 验证 (尝试 {attempt}/{NUM_TRIES})")
    print("=" * 60)

    processed_code = write_lean(current)
    ok, last_log = lake_build()

    has_sorry = "sorry" in processed_code.lower()

    if ok and not has_sorry:
        print(" 验证通过！")
        break
    elif ok and has_sorry:
        print("⚠ lake build 通过但代码中仍有 sorry")
        ok = False
    else:
        print("✗ lake build 失败")

    brief = brief_errors(last_log)
    print()
    print("【关键报错】")
    print(brief[:800])

    with open(os.path.join(jobdir, f"build_log_{attempt}.txt"), "w", encoding="utf-8") as f:
        f.write(last_log)

    if attempt >= NUM_TRIES:
        print()
        print(f"⚠ 已达到最大重试次数 ({NUM_TRIES})")
        break

    print()
    print("=" * 60)
    print(f"【第三步】自动修复 (尝试 {attempt+1})")
    print("=" * 60)

    t0 = time.time()
    fix_response = generate_proof(processed_code, is_fix=True, error_msg=brief)
    print(f"✓ 修复生成完成，用时 {time.time()-t0:.1f}s")

    current = extract_lean_block(fix_response)

    with open(os.path.join(jobdir, f"full_response_{attempt+1}.txt"), "w", encoding="utf-8") as f:
        f.write(fix_response)
    with open(os.path.join(jobdir, f"candidate_{attempt+1}.lean"), "w", encoding="utf-8") as f:
        f.write(current + "\n")

# 保存最终结果
with open(os.path.join(jobdir, "final_Demo.lean"), "w", encoding="utf-8") as f:
    f.write(current + "\n")

print()
print("=" * 60)
print("【最终结果】")
print("=" * 60)
print("输出文件:", OUT_LEAN)
print("实验目录:", jobdir)
if ok:
    print(" 状态: 成功 - 证明已通过验证！")
else:
    print(" 状态: 失败 - 未通过验证")


Lean 4 自动证明系统
Lean 工程目录: /content/lean_ds/ds_demo
输出文件路径: /content/lean_ds/ds_demo/DsDemo/Demo.lean

【预处理】转换输入符号
✓ 输入无需转换

【第一步】生成证明计划与代码
✓ 生成完成，用时 22.4s

【证明计划 (Proof Plan)】
----------------------------------------


### Detailed Proof and Analysis

#### 1. Problem Understanding
The theorem states that for any real number `x > 0`, the inequality `x + 1 > 1` holds.  

#### 2. Mathematical Background
This is a simple inequality involving addition and subtraction.  

#### 3. Proof Strategy
We need to prove that `x + 1 > 1` given that `x > 0`.  

First, observe that the inequality can be rewritten as `x > 0` (by subtracting 1 from both sides).  
But we already know that `x > 0` is given.  
Thus, `x + 1 > 0 + 1 = 1`, which simplifies to `x + 1 > 1`.  

Alternatively, we can directly reason as follows:  
Since `x > 0`, we can add `1` to both sides of the inequality to get `x + 1 > 0 + 1 = 1`, which is exactly the desired result.  

#### 4. Step-by-Step Reasoning
1. We start with `x > 0`.  
2. We add `1` to both sides of the inequality `x > 0` to get `x + 1 > 1`.  
   - This is valid because adding the same quantity (`1`) to both sides of an inequality preserves the inequality.  

#### 5. Lean 4 Tactics
We can use the `linarith` tactic to handle this inequality, but since the problem is straightforward, we can also use the `norm_num` tactic to simplify the goal.  

However, `linarith` is a more general tactic that can handle linear arithmetic, so it might be overkill here.  

#### 6. Lean 4 Proof Sketch (using `have` statements)

----------------------------------------

【第二步】lake build 验证 (尝试 1/3)
⚠ lake build 通过但代码中仍有 sorry

【关键报错】
✔ [7888/7892] Built DsDemo.Basic (297ms)
⚠ [7890/7892] Built DsDemo.Demo (5.7s)
✔ [7891/7892] Built DsDemo (5.4s)
Build completed successfully (7892 jobs).


【第三步】自动修复 (尝试 2)
✓ 修复生成完成，用时 3.9s

【第二步】lake build 验证 (尝试 2/3)
 验证通过！

【最终结果】
输出文件: /content/lean_ds/ds_demo/DsDemo/Demo.lean
实验目录: /content/ds_prover_runs/my_experiment_1769503401
 状态: 成功 - 证明已通过验证！


# 第四部分：结果分析

In [21]:
#@title 4.1 查看证明计划与代码
#@markdown 展示 AI 生成的证明计划和最终代码。
#@markdown 如果证明计划是英文，会自动翻译成中文。


import os
import re
import torch
from IPython.display import display, Markdown

# ╔══════════════════════════════════════════════════════════════════╗
# ║                    文本处理工具函数                               ║
# ╚══════════════════════════════════════════════════════════════════╝

def fix_latex_to_readable(text):
    """将 LaTeX 公式转换成易读格式"""
    if not text:
        return text

    # 分数处理
    for _ in range(5):
        old_text = text
        text = re.sub(r'\\frac\s*\{([^{}]+)\}\s*\{([^{}]+)\}', r'((\1)/(\2))', text)
        if old_text == text:
            break

    text = re.sub(r'\(\((\d+)\)\)', r'\1', text)
    text = re.sub(r'\((\d+)\)(?!/)', r'\1', text)

    replacements = {
        r'\left|': '|', r'\right|': '|',
        r'\left(': '(', r'\right)': ')',
        r'\left[': '[', r'\right]': ']',
        r'\lvert': '|', r'\rvert': '|', r'\vert': '|',
        r'\times': '\u00d7', r'\cdot': '\u00b7', r'\div': '\u00f7', r'\pm': '\u00b1',
        r'\leq': '\u2264', r'\geq': '\u2265', r'\neq': '\u2260',
        r'\le': '\u2264', r'\ge': '\u2265', r'\ne': '\u2260',
        r'\approx': '\u2248', r'\equiv': '\u2261',
        r'\forall': '\u2200', r'\exists': '\u2203',
        r'\land': '\u2227', r'\lor': '\u2228', r'\neg': '\u00ac',
        r'\to': '\u2192', r'\rightarrow': '\u2192', r'\leftarrow': '\u2190',
        r'\Rightarrow': '\u21d2', r'\Leftarrow': '\u21d0', r'\Leftrightarrow': '\u21d4',
        r'\in': '\u2208', r'\notin': '\u2209',
        r'\subset': '\u2282', r'\subseteq': '\u2286',
        r'\cup': '\u222a', r'\cap': '\u2229', r'\emptyset': '\u2205',
        r'\mathbb{R}': '\u211d', r'\mathbb{Q}': '\u211a',
        r'\mathbb{Z}': '\u2124', r'\mathbb{N}': '\u2115',
        r'\R': '\u211d', r'\Q': '\u211a', r'\Z': '\u2124', r'\N': '\u2115',
        r'\alpha': '\u03b1', r'\beta': '\u03b2', r'\gamma': '\u03b3',
        r'\delta': '\u03b4', r'\epsilon': '\u03b5', r'\theta': '\u03b8',
        r'\lambda': '\u03bb', r'\mu': '\u03bc', r'\pi': '\u03c0',
        r'\sigma': '\u03c3', r'\phi': '\u03c6', r'\omega': '\u03c9',
        r'\infty': '\u221e', r'\partial': '\u2202', r'\nabla': '\u2207',
        r'\sum': '\u2211', r'\prod': '\u220f', r'\int': '\u222b', r'\sqrt': '\u221a',
        r'\[': '', r'\]': '', r'\quad': '  ', r'\qquad': '    ',
    }

    for latex, replacement in replacements.items():
        text = text.replace(latex, replacement)

    text = re.sub(r'\^{([^{}]+)}', r'^(\1)', text)
    text = re.sub(r'\_\{([^{}]+)\}', r'_(\1)', text)
    text = re.sub(r'\\text\{([^{}]+)\}', r'\1', text)
    text = re.sub(r'\\textbf\{([^{}]+)\}', r'**\1**', text)
    text = re.sub(r'\\mathrm\{([^{}]+)\}', r'\1', text)
    text = re.sub(r'\\operatorname\{([^{}]+)\}', r'\1', text)
    text = re.sub(r'\$\$([^$]+)\$\$', r'\1', text)
    text = re.sub(r'\$([^$]+)\$', r'\1', text)
    text = re.sub(r'\\([a-zA-Z]+)\s*', r'', text)
    text = re.sub(r'  +', ' ', text)

    return text.strip()


def is_mostly_english(text):
    """检测文本是否主要是英文"""
    if not text:
        return False
    chinese_chars = sum(1 for c in text if '\u4e00' <= c <= '\u9fff')
    ascii_letters = sum(1 for c in text if c.isascii() and c.isalpha())
    total_letters = chinese_chars + ascii_letters
    if total_letters == 0:
        return False
    return ascii_letters / total_letters > 0.7


def translate_to_chinese_full(text, model, tok):
    """
    完整翻译英文证明计划为中文
    增强版：更强的 prompt + 更多 tokens
    """

    translate_prompt = """你是一个专业的数学翻译。请将下面的英文数学证明计划完整翻译成中文。

【翻译要求】
1. 所有英文解释性文字必须翻译成中文
2. 数学公式和表达式保持不变
3. Lean 代码块保持不变（```lean 或 ```lean4 之间的内容）
4. Lean 策略名保持英文，如 simp, norm_num, ring, linarith, exact 等
5. 保持原有的结构、编号和格式
6. "Proof Sketch" 翻译为 "证明草图"
7. "Explanation" 翻译为 "说明"
8. "have" 语句的解释要翻译

【需要翻译的文本】
```
""" + text + """
```

"""

    chat = [{"role": "user", "content": translate_prompt}]
    inputs = tok.apply_chat_template(
        chat, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    )

    # 兼容不同版本的 transformers
    if isinstance(inputs, torch.Tensor):
        input_ids = inputs.to(model.device)
    else:
        input_ids = inputs["input_ids"].to(model.device)

    attention_mask = torch.ones_like(input_ids)

    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=4000,  # 增加到 4000
            do_sample=False,
            temperature=1.0,
            pad_token_id=tok.eos_token_id,
        )

    full_output = tok.decode(out[0], skip_special_tokens=True)
    input_text = tok.decode(input_ids[0], skip_special_tokens=True)
    translated = full_output[len(input_text):].strip()

    # 清理开头结尾
    translated = re.sub(r'^[\"\'\s`]+', '', translated)
    translated = re.sub(r'[\"\'\s`]+$', '', translated)

    # 如果翻译结果太短，可能失败了
    if len(translated) < len(text) * 0.3:
        print("[警告] 翻译结果过短，可能不完整")

    return translated


def split_and_translate(text, model, tok):
    """
    分段翻译：将文本分成多段分别翻译，确保完整
    """
    # 按段落分割
    paragraphs = re.split(r'\n\n+', text)

    translated_parts = []

    for i, para in enumerate(paragraphs):
        para = para.strip()
        if not para:
            continue

        # 如果是代码块，保持不变
        if para.startswith('```') or para.startswith('theorem ') or para.startswith('import '):
            translated_parts.append(para)
            continue

        # 如果已经是中文，保持不变
        if not is_mostly_english(para):
            translated_parts.append(para)
            continue

        # 翻译这一段
        try:
            short_prompt = """Translate this English text to Chinese. Keep code and math formulas unchanged:

""" + para + """

Chinese:"""

            chat = [{"role": "user", "content": short_prompt}]
            input_ids = tok.apply_chat_template(
                chat, tokenize=True, add_generation_prompt=True, return_tensors="pt"
            ).to(model.device)

            with torch.no_grad():
                out = model.generate(
                    input_ids=input_ids,
                    max_new_tokens=1000,
                    do_sample=False,
                    pad_token_id=tok.eos_token_id,
                )

            full_output = tok.decode(out[0], skip_special_tokens=True)
            input_text = tok.decode(input_ids[0], skip_special_tokens=True)
            translated_para = full_output[len(input_text):].strip()
            translated_para = re.sub(r'^[\"\'\s`]+', '', translated_para)
            translated_para = re.sub(r'[\"\'\s`]+$', '', translated_para)

            translated_parts.append(translated_para if translated_para else para)

        except Exception as e:
            translated_parts.append(para)

    return '\n\n'.join(translated_parts)


def translate_proof_plan(text, model, tok, method='full'):
    """
    翻译证明计划
    method: 'full' = 整体翻译, 'split' = 分段翻译
    """
    if method == 'split':
        return split_and_translate(text, model, tok)
    else:
        return translate_to_chinese_full(text, model, tok)


def extract_proof_plan_safe(full_response):
    """安全地提取证明计划"""
    if not full_response:
        return ""

    patterns = [r"```lean4", r"```lean", r"```"]
    for pattern in patterns:
        match = re.search(pattern, full_response)
        if match:
            return full_response[:match.start()].strip()
    return full_response.strip()


# ╔══════════════════════════════════════════════════════════════════╗
# ║                         主程序                                    ║
# ╚══════════════════════════════════════════════════════════════════╝

# 翻译方法选择
TRANSLATE_METHOD = 'full'

_missing = [v for v in ['jobdir', 'OUT_LEAN'] if v not in globals()]
if _missing:
    print("请先运行第三部分（缺少变量: {})".format(', '.join(_missing)))
else:
    print("=" * 60)
    print("结果概览")
    print("=" * 60)

    if 'ok' in globals():
        status = "SUCCESS - 证明已通过验证" if ok else "FAILED - 未通过验证"
    else:
        status = "未知（请先运行验证）"
    print("状态:", status)
    print("输出文件:", OUT_LEAN)
    print("实验目录:", jobdir)
    print()

    # ===== 显示证明计划 =====
    first_resp_path = os.path.join(jobdir, "full_response_1.txt")

    if os.path.exists(first_resp_path):
        with open(first_resp_path, "r", encoding="utf-8") as f:
            full_response = f.read()

        if 'extract_proof_plan' in dir():
            proof_plan = extract_proof_plan(full_response)
        else:
            proof_plan = extract_proof_plan_safe(full_response)

        if proof_plan:
            print("=" * 60)
            print("证明计划 (Proof Plan)")
            print("=" * 60)

            # Step 1: 转换 LaTeX
            processed_plan = fix_latex_to_readable(proof_plan)

            # Step 2: 翻译
            try:
                if TRANSLATE_METHOD != 'none' and 'model' in globals() and 'tok' in globals():
                    if is_mostly_english(processed_plan):
                        print("[正在翻译成中文，请稍候...]")
                        translated_plan = translate_proof_plan(
                            processed_plan, model, tok, method=TRANSLATE_METHOD
                        )

                        # 检查翻译质量
                        if is_mostly_english(translated_plan):
                            print("[警告：翻译可能不完整，尝试分段翻译...]")
                            translated_plan = translate_proof_plan(
                                processed_plan, model, tok, method='split'
                            )

                        processed_plan = translated_plan
                        print("[已翻译成中文]")
                    else:
                        print("[原文已是中文]")
                else:
                    print("[跳过翻译]")

                print("-" * 60)
                display(Markdown(processed_plan))
                print()

            except Exception as e:
                print("处理时出错:", str(e))
                print("显示原始内容：")
                print("-" * 60)
                display(Markdown(proof_plan))
                print()
        else:
            print("未能提取证明计划")
            print()
    else:
        print("未找到证明计划文件")
        print()

    # ===== 显示最终代码 =====
    print("=" * 60)
    print("最终证明代码")
    print("=" * 60)

    demo_code = ""
    if os.path.exists(OUT_LEAN):
        with open(OUT_LEAN, "r", encoding="utf-8") as f:
            demo_code = f.read().strip()
    elif 'current' in globals() and current:
        demo_code = current.strip()

    if demo_code:
        if "sorry" in demo_code.lower():
            print("[注意：代码中仍包含 sorry]")
        display(Markdown("```lean\n{}\n```".format(demo_code[:15000])))
    else:
        print("未找到证明代码")

    print()
    print("=" * 60)

结果概览
状态: SUCCESS - 证明已通过验证
输出文件: /content/lean_ds/ds_demo/DsDemo/Demo.lean
实验目录: /content/ds_prover_runs/my_experiment_1769503401

证明计划 (Proof Plan)
[正在翻译成中文，请稍候...]
[已翻译成中文]
------------------------------------------------------------


### 翻译

#### 1. 问题理解
定理指出对于任何实数 `x > 0`，不等式 `x + 1 > 1` 成立。

#### 2. 数学背景
这是一个简单的涉及加法和减法的不等式。

#### 3. 证明策略
我们需要证明 `x + 1 > 1`，给定 `x > 0`。

首先，观察不等式可以重写为 `x > 0`（通过从两边减去 1）。
但我们已经知道 `x > 0` 是给定的。
因此，`x + 1 > 0 + 1 = 1`，这简化为 `x + 1 > 1`。

或者，我们可以直接这样推理：
由于 `x > 0`，我们在不等式 `x > 0` 的两边都加上 `1` 得到 `x + 1 > 0 + 1 = 1`，这就是所需要的结论。

#### 4. 逐步推理
1. 我们从 `x > 0` 开始。
2. 我们在不等式 `x > 0` 的两边都加上 `1` 得到 `x + 1 > 1`。
   - 这是有效的，因为对不等式的两边加上相同的量（1）会保持不等式。

#### 5. Lean 4 策略
我们可以使用 `linarith` 策略来处理这个不等式，但是由于这个问题很简单，我们也可以使用 `norm_num` 策略来简化目标。

但是 `linarith` 是一个更通用的策略，可以处理线性算术，所以它可能在这里有点大材小用。

#### 6. Lean 4 证明草图（使用 `have` 语句）
```lean4
theorem simple_inequality (x : ℝ) (hx : x > 0) : x + 1 > 1 := by
  have h : x + 1 > 1 := by
    -- 从 `x > 0` 开始，在两边都加上 1
    linarith
  exact h
```

#### 7. 解释
- `have h : x + 1 > 1 := by linarith` 表示我们使用 `linarith` 策略来证明 `x + 1 > 1`，因为 `x > 0` 是已知的，`linarith` 可以处理这种线性不等式。
- `exact h` 表示我们已经证明了 `x + 1 > 1`，所以可以结束证明。

#### 8. 翻译
- `have` 语句的解释：我们有一个 `have` 语句，它表示我们有一个中间结果 `h`，即 `x + 1 > 1`，这个结果是通过 `linarith` 策略从 `x > 0` 推导出来的。

### 完整翻译

```lean4
theorem simple_inequality (x : ℝ) (hx : x > 0) : x + 1 > 1 := by
  have h : x + 1 > 1 := by
    -- 从 `x > 0` 开始，在两边都加上 1
    linarith
  exact h


最终证明代码


```lean
import Mathlib

theorem simple_ineq (x : ℝ) (h : x > 0) : x + 1 > 1 := by
  have h_main : x + 1 > 1 := by
    -- We need to show that x + 1 > 1.
    -- Given x > 0, we can use this fact to prove the inequality.
    linarith
  -- The main inequality has been proven, so we can conclude the theorem.
  exact h_main
```

In [22]:
#@title 4.2 查看实验文件
#@markdown 列出本次实验生成的所有文件：
#@markdown - candidate_*.lean: 每次生成的候选证明
#@markdown - full_response_*.txt: AI 的完整输出（含证明计划）
#@markdown - build_log_*.txt: lake build 日志
#@markdown - final_Demo.lean: 最终结果

import os
import glob

if 'jobdir' not in globals():
    print("请先运行第二部分和第三部分")
else:
    print("实验目录:", jobdir)
    print()
    files = sorted(glob.glob(os.path.join(jobdir, "*")))

    if not files:
        print("目录为空，请先运行第三部分")
    else:
        print(f"共 {len(files)} 个文件:")
        print()
        for p in files:
            size = os.path.getsize(p)
            print(f"  {os.path.basename(p):35s} ({size:,} bytes)")

    # 显示最后一次构建日志
    logs = sorted(glob.glob(os.path.join(jobdir, "build_log_*.txt")))
    if logs:
        last = logs[-1]
        print()
        print("=" * 50)
        print(f"最后一次构建日志 ({os.path.basename(last)}) 末尾 40 行:")
        print("=" * 50)
        with open(last, "r", encoding="utf-8") as f:
            tail = f.read().splitlines()[-40:]
        print("\n".join(tail))

实验目录: /content/ds_prover_runs/my_experiment_1769503401

共 8 个文件:

  build_log_1.txt                     (219 bytes)
  candidate_1.lean                    (106 bytes)
  candidate_2.lean                    (299 bytes)
  final_Demo.lean                     (299 bytes)
  full_response_1.txt                 (1,902 bytes)
  full_response_2.txt                 (311 bytes)
  input_original.txt                  (181 bytes)
  input_processed.txt                 (181 bytes)

最后一次构建日志 (build_log_1.txt) 末尾 40 行:
✔ [7888/7892] Built DsDemo.Basic (297ms)
⚠ [7890/7892] Built DsDemo.Demo (5.7s)
✔ [7891/7892] Built DsDemo (5.4s)
Build completed successfully (7892 jobs).



# 第五部分：导出结果

## 参数说明：下载选项

| 参数 | 说明 |
|------|------|
| `download_zip` | 下载整个实验目录（打包为 zip） |
| `download_demo` | 只下载最终证明文件 |

注意：仅在 Colab 环境下会弹出下载框。本地 Jupyter 请从左侧文件面板手动下载。

In [23]:
#@title 5.1 下载实验结果

download_zip = True   #@param {type:"boolean"}
download_demo = True  #@param {type:"boolean"}

import os
import subprocess

if 'OUT_LEAN' not in globals() or 'jobdir' not in globals():
    print("请先运行第三部分")
else:
    zip_path = os.path.join(jobdir, "experiment_results.zip")

    if download_zip:
        cmd = f'cd "{jobdir}" && zip -q -r experiment_results.zip .'
        subprocess.run(["bash", "-c", cmd], check=False)

    try:
        from google.colab import files
        IS_COLAB = True
    except ImportError:
        IS_COLAB = False

    if not IS_COLAB:
        print("非 Colab 环境，请在左侧文件面板手动下载")
        print("文件位置:", jobdir)
    else:
        if download_demo and os.path.exists(OUT_LEAN):
            print("下载 Demo.lean ...")
            files.download(OUT_LEAN)

        if download_zip and os.path.exists(zip_path):
            print("下载 experiment_results.zip ...")
            files.download(zip_path)

下载 Demo.lean ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

下载 experiment_results.zip ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 附录

## 常见问题

| 问题 | 可能原因 | 解决方案 |
|------|----------|----------|
| CUDA out of memory | 显存不足 | 降低 MAX_NEW_TOKENS 到 1200 |
| lake build 失败 | 定理过于复杂 | 尝试更简单的定理，或增加 NUM_TRIES |
| 生成的代码仍有 sorry | 模型无法完成证明 | 提高 TEMPERATURE，或换用更强的模型 |
| 没有看到证明计划 | 输出格式问题 | 查看 full_response_1.txt 完整内容 |

## 参考资料

- 论文：Ren et al. [DeepSeek-Prover-V2: Advancing Formal Mathematical Reasoning via Reinforcement Learning for Subgoal Decomposition](https://arxiv.org/abs/2504.21801). arXiv 2025
- 代码：https://github.com/deepseek-ai/DeepSeek-Prover-V2
- 模型：https://huggingface.co/deepseek-ai/DeepSeek-Prover-V2-7B
- Lean 4 在线编辑器：https://leanprover.github.io/lean4web/
- ProverBench 数据集：https://huggingface.co/datasets/deepseek-ai/DeepSeek-ProverBench
- Lean 4 官方文档：https://leanprover.github.io/lean4/doc/
- Mathlib 文档：https://leanprover-community.github.io/mathlib4_docs/